# Module 4: Content-based filtering

In de vorige module hebben we gekeken naar _collaborative filtering_. Een van de belangrijkste problemen die we daarmee zijn tegengekomen is de _long tail_. Er is vaak niet genoeg interactie-data om items of users goed met elkaar te kunnen vergelijken. In de praktijk wordt er daarom vaak gebruik gemaakt van _content-based filtering_.

Content-based filtering refereert niet aan één type algoritme. Er zijn enorm veel verschillende aanpakken mogelijk. Het is onmogelijk om in het tijdsbestek van dit vak een volledig overzicht te geven. We zoomen in deze module daarom in deze module op een paar veelgebruikte technieken. 

Deze module staat uit ~~4~~~3 losse submodules:

* Module 4a: In het eerste deel ga je leren hoe goed we aanbevelingen kunnen doen aan de hand van filmgenres. Het genre geeft zeer oppervlakkige informatie over de inhoud van de film, maar misschien is het een zinvolle manier van films vergelijken. En, het geeft ons in ieder geval een goede baseline voor complexere aanpakken.

* Module 4b: In dit deel ga je leren hoe je _natural language processing (NLP)_ kan gebruiken voor content-based filtering. Je gaat leren wat *tf-idf vectorization* is en hoe je dat kan gebruiken voor vergelijken van films aan de hand van de ondertiteling. De ondertiteling geeft zeer gedetaileerde informatie over de inhoud van een film.

* Module 4c: Hier ga je verder met NLP aan de hand van SpaCy. SpaCy is een moderne Python library voor het analyseren van tekst.

* ~~Module 4d: Je gaat leren wat Decision Trees zijn, en hoe je die kan gebruiken om nieuwe features te generen voor films.~~~


# Module 4a: Content-based filtering using genre

**Categorical features and Jaccard index**

Het is altijd verstandig om zo eenvoudig mogelijk te beginnen. Het is heel makkelijk om een verschrikkelijk complex systeem te maken waarin we gaan kijken hoe in welke mate films op elkaar lijken aan de hand van alle informatie die op IMDB staat. Maar misschien werkt iets veel eenvoudigers wel veel beter.

De data die we hebben bevat informatie over de genres. En het is redelijk eenvoudig om films te vergelijken aan de hand van de genres. Dus dit is een goede plek om te beginnen.


Begin met het laden van de libraries en collaborative filtering functies van de vorige modules.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# course specific packages
import answers

# import from previous module
from helpers import number_of_movies, number_of_users, number_of_ratings, rating_density, split_data
from cf1 import pivot_ratings, fill_nan_mean, create_similarity_matrix_cosine,\
                mean_center_columns, select_neighborhood, weighted_mean, mean_center_rows
from cf2 import predict_ratings


Nu kunnen we de data inladen. We gebruiken de dataset van module 1, deze is met ruim 100.000 ratings veel uitgebreider dan de dataset van de vorige opdracht.

In [2]:
datapath = "./data/ml-latest-small"

df_ratings = pd.read_csv(f"{datapath}/ratings.csv")
df_ratings_training, df_ratings_test = split_data(df_ratings, d=0.9)
actual = df_ratings_test[['userId', 'movieId', 'rating']]

print(f"ratings header    : {list(df_ratings_training.columns)}")
print(f"number of movies  : {number_of_movies(df_ratings_training)}")
print(f"number of users   : {number_of_users(df_ratings_training)}")
print(f"number of ratings : {number_of_ratings(df_ratings_training)}")
print(f"density of ratings: {rating_density(df_ratings_training)}")

ratings header    : ['userId', 'movieId', 'rating', 'timestamp']
number of movies  : 9387
number of users   : 610
number of ratings : 90730
density of ratings: 0.01584507349718044


## Baseline

We beginnen eerst met het vaststellen van een baseline. Hoe goed werkt de beste aanpak uit de vorige opdracht (item based collaborative filtering) met deze dataset? De density van de data is erg laag, dus we verwachten geen geweldige prestaties. Desondanks moeten we dit proberen omdat we anders niet kunnen vaststellen hoe verdere aanpakken zich verhouden tot wat we eerder hebben gedaan. 

Eerst laden we de functies voor het berekenen van de utility en similarity matrix.

Bereken de utility en similarity matrix.

In [3]:
df_utility_ratings = pivot_ratings(df_ratings_training)
df_utility_ratings_filled = fill_nan_mean(df_utility_ratings)
df_similarity_ratings = create_similarity_matrix_cosine(df_utility_ratings_filled)
display(df_utility_ratings.head())
display(df_similarity_ratings.head())

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
movieId,,,,,,,,,,,,,,,,,,,,,
1,4.0,NaN,NaN,NaN,4.0,NaN,4.5,NaN,NaN,NaN,...,4.0,NaN,4.0,3.0,4.0,2.5,4.0,2.5,NaN,5.0
2,NaN,NaN,NaN,NaN,NaN,4.0,NaN,4.0,NaN,NaN,...,NaN,4.0,NaN,5.0,3.5,NaN,NaN,2.0,NaN,NaN
3,4.0,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


movieId,1,2,3,4,5,6,7,8,9,10,...,191005,193565,193567,193571,193573,193579,193583,193585,193587,193609
movieId,,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.990014,0.990275,0.992857,0.990861,0.990215,0.990217,0.992902,0.992519,0.987204,...,0.993342,0.993342,0.993342,0.993342,0.993342,0.993342,0.993342,0.993342,0.993342,0.993342
2,0.990014,1.000000,0.992455,0.993984,0.992692,0.992085,0.991386,0.993943,0.993953,0.988439,...,0.994693,0.994693,0.994693,0.994693,0.994693,0.994693,0.994693,0.994693,0.994693,0.994693
3,0.990275,0.992455,1.000000,0.995296,0.994280,0.993803,0.993405,0.995582,0.995213,0.989554,...,0.995694,0.995694,0.995694,0.995694,0.995694,0.995694,0.995694,0.995694,0.995694,0.995694
4,0.992857,0.993984,0.995296,1.000000,0.996396,0.996108,0.995426,0.998493,0.998340,0.993757,...,0.999358,0.999358,0.999358,0.999358,0.999358,0.999358,0.999358,0.999358,0.999358,0.999358
5,0.990861,0.992692,0.994280,0.996396,1.000000,0.993878,0.994522,0.996691,0.995766,0.990978,...,0.996742,0.996742,0.996742,0.996742,0.996742,0.996742,0.996742,0.996742,0.996742,0.996742


Vervolgens laden we de `predict_ratings` functie hieronder. Deze functie geeft een neighborhood based voorspelling voor test data gegeven een similarity en een utility matrix.

Voer de voorspelling uit. Deze berekening kan zeker wel een minuut duren.

In [4]:
df_predicted_cf_item_based = predict_ratings(df_similarity_ratings, df_utility_ratings, actual)

In [5]:
display(df_predicted_cf_item_based.head())

,userId,movieId,predicted rating
3,1,47,4.369164
27,1,500,4.369134
28,1,527,4.369258
39,1,673,4.369114
43,1,804,4.369199


En vervolgens gebruiken we weer de mean squared error voor het evalueren van de voorspellingen.

In [6]:
def mse(actual, predicted_ratings):
    """Computes the mean square error between actual ratings and predicted ratings
    
    Arguments:
    predicted_ratings -- a dataFrame containing the columns rating and predicted rating
    """
    diff = actual['rating'] - predicted_ratings['predicted rating']
    return (diff**2).mean()

In [7]:
mse_cf_item_based = mse(actual, df_predicted_cf_item_based)
print(f'mse for item based collaborative filtering {mse_cf_item_based:.2f}')

mse for item based collaborative filtering 1.27


Deze score kunnen we niet vergelijken met de uitkomsten van de vorige opdracht omdat dit weer een andere dataset is, en we de functies net wat anders hebben geïmplementeerd. Toch kunnen we wel een eerste intuititie vormen over hoe goed deze score is. Is de uitkomst beter dan zo maar wat gokken?

### Vraag 1

\[1 pt.\]

Genereer random voorspellingen voor de test set. Dus gebruik niet de functie `predict_ratings`, maar genereer random "voorspellingen" tussen de 0.5 en de 5. Bereken vervolgens de mean squared error voor deze random ratings en stop de uitkomst in de variabele `mse_random`.

In [8]:
import random

mse_random = 0
# TODO

display(actual)
actuals = actual["rating"].reset_index(drop=True)

rating_pred =[]
for i in range(len(actuals)):
    x = random.uniform(0.5, 5.0)
    rating_pred.append(x)
    mse_random += (actuals[i]-x)**2

mse_random =(1/len(actuals))*mse_random
    
print(f'mse for random prediction: {mse_random:.2f}')

,userId,movieId,rating
3,1,47,5.0
27,1,500,3.0
28,1,527,5.0
39,1,673,3.0
43,1,804,4.0
...,...,...,...
100799,610,147657,4.0
100803,610,148626,4.0
100808,610,152081,4.0
100809,610,152372,3.5


mse for random prediction: 3.25


In [9]:
answers.test_1(mse_random)

Testing mse value: success!


Dus voorspellen aan de hand van item based collaborative filtering doet het beter dan volstrekte willekeur. Dat is een goed begin. 

Maar we hebben meer informatie over de data. Wat is bijvoorbeeld de gemiddelde rating in de training set?

### Vraag 2

\[1 pt.\]

Berken de gemiddelde rating in de training set. Genereer een "voorspelling" voor de test set waarin je elke film deze gemiddelde rating toekent (elke film krijgt dus exact dezelfde voorspelde rating). Bereken weer de mse en sla de uitkomst op in `mse_mean`.

In [10]:
mse_mean = 0

predicted_item_mean = df_ratings_test[['userId', 'movieId', 'rating']]
item_mean= predicted_item_mean.groupby('movieId').agg({'rating': np.mean})
item_mean["predicted rating"] = item_mean["rating"]

item_mean = item_mean.loc[:, item_mean.columns != "rating"]
predicted_item_mean =predicted_item_mean.join(item_mean, on= "movieId")

mse_mean = sum((predicted_item_mean["rating"] - predicted_item_mean["predicted rating"])**2)
    
mse_mean =(1/len(predicted_item_mean))*mse_mean
# TODO

print(f'mse for mean prediction: {mse_mean:.2f}')

mse for mean prediction: 0.53


In [11]:
answers.test_2(mse_mean, mse_cf_item_based)

Testing mse value: success!


### Vraag 3

\[1 pt.\]

Wat is de conclusie? Geeft item based collaborative filtering goede voorspellingen?

We kunnen concluderen dat op items gebaseerde collaboratieve filtering beter presteert dan willekeurige voorspelling, maar niet zo goed als mean prediction.

De MSE van 1.27 voor op items gebaseerde collaboratieve filtering is lager dan de MSE van random prediction 3.37, wat aangeeft dat op item based collaborative filtering betere voorspellingen doet dan een random prediction. De MSE van 1.27 is echter hoger dan de MSE van gemiddelde voorspelling 0.53, wat aangeeft dat op item based collaborative filtering niet zo goed presteert alsmean prediction

### Vraag 4

\[1 pt.\]

Verzin zelf een item-based collaborative filtering strategie waarvan je denkt dat die beter werkt dan de methodes die gegeven zijn in de vorige drie vragen en leg in een paar zinnen uit waarom je denkt dat die beter werkt (je hoeft geen implementatie te geven).

YOUR ANSWER HERE

## Genres

Laten we nu eens kijken of we het beter kunnen krijgen met een relatief eenvoudige aanpak. Kunnen we voorspellingen doen aan de hand van filmgenres? Met andere woorden wat gebeurt er als we ervan uitgaan dat films met veel overlap in genres ook sterk op elkaar lijken?

Eerst moeten we de genres verzamelen. De genres zitten in de file movies.csv, maar zoals je je misschien nog kan herinneren van opdracht 1 zijn deze op een onhandige manier opgeslagen. Laad hieronder de functies voor het handig converteren vand deze data.

In [12]:
def extract_genres(movies):
    """Create an unfolded genre dataframe. Unpacks genres seprated by a '|' into seperate rows.

    Arguments:
    movies -- a dataFrame containing at least the columns 'movieId' and 'genres' 
              where genres are seprated by '|'
    """
    genres_m = movies.apply(lambda row: pd.Series([row['movieId']] + row['genres'].lower().split("|")), axis=1)
    stack_genres = genres_m.set_index(0).stack()
    df_stack_genres = stack_genres.to_frame()
    df_stack_genres['movieId'] = stack_genres.index.droplevel(1)
    df_stack_genres.columns = ['genre', 'movieId']
    return df_stack_genres.reset_index()[['movieId', 'genre']]

def pivot_genres(df):
    """Create a one-hot encoded matrix for genres.
    
    Arguments:
    df -- a dataFrame containing at least the columns 'movieId' and 'genre'
    
    Output:
    a matrix containing '0' or '1' in each cell.
    1: the movie has the genre
    0: the movie does not have the genre
    """
    return df.pivot_table(index = 'movieId', columns = 'genre', aggfunc = 'size', fill_value=0)

Laad de data. Dit kan even duren, de functie `extract_genres` kost wat tijd. 

In [13]:
df_movies = pd.read_csv(f"{datapath}/movies.csv")
df_genres = extract_genres(df_movies)

print(f"movies header: {list(df_movies.columns)}")
print(f"genres header: {list(df_genres.columns)}")

display(df_genres.head())

movies header: ['movieId', 'title', 'genres']
genres header: ['movieId', 'genre']


,movieId,genre
0,1,adventure
1,1,animation
2,1,children
3,1,comedy
4,1,fantasy


Bereken nu de utility matrix voor genres. In deze matrix zijn de genres de features en geeft voor elk film aan onder welke genres deze valt. Een `1` betekent dat de film onder het genre valt. Dus bij in de onderstaande tabel kunnen we zien dat film `4` onder de genres `comedy`, `drama` en `romance` valt.

Het gebruiken van `1`/`0` voor het coderen van categorische data, wordt ook wel **one-hot encoding** genoemd.

In [14]:
df_utility_genres = pivot_genres(df_genres)
display(df_utility_genres.head())

genre,(no genres listed),action,adventure,animation,children,comedy,crime,documentary,drama,fantasy,film-noir,horror,imax,musical,mystery,romance,sci-fi,thriller,war,western
movieId,,,,,,,,,,,,,,,,,,,,
1,0,0,1,1,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0
2,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0
4,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0
5,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


Aan de hand van deze utility matrix kunnen we weer een similarity matrix maken. Dit kunnen we helaas niet doen met cosine similarity omdat deze features geen numerieke betekenis hebben. Het zijn _categorische features_: de nummers geven aan of de film onder een bepaalde categorie (genre) valt of niet. Voor het bepalen van similiarities voor catagorische data zijn er een legioen aan mogelijke maten$^1$. Voor deze opdracht gebruiken we de [**Jaccard index**](https://en.wikipedia.org/wiki/Jaccard_index). Dit is een veelgebruikte maat die goed werkt voor binaire data en die bovendien eenvoudig te implementeren is.

Om de Jaccard index voor twee films ($A$ en $B$) te bereken moeten we drie waardes bepalen:
- *M11*: Het aantal features dat in de matrix voor zowel film $A$ als $B$ een $1$ bevat. Oftewel, het aantal genres waarin $A$ en $B$ overlappen:
- *M10*: Het aantal features dat in de matrix voor alléén film $A$ een $1$ bevat. Oftewel, het aantal genres dat *wel* op $A$ van toepassing is en *niet* op $B$.
- *M01*: Het aantal features dat in de matrix voor alléén film $B$ een $1$ bevat.

De berekening van de Jaccard index is vervolgens:

$$
\textrm{jaccard}(A, B)=\frac{\textrm{M11}}{\textrm{M10} + \textrm{M01} - \textrm{M11}}
$$

Of in set-notatie (als je daar bekend mee bent):

$$
\textrm{jaccard}(A, B)=\frac{|A \cap B|}{|A \cup B| - |A \cap B|}
$$ 

<p style="font-size:9pt;">
    1: Voor een overzicht kan je even naar <a href="https://pdfs.semanticscholar.org/c654/4a12fec2097bddc49adbb159426d9dc15d2c.pdf">[Boriah 2007]</a> kijken. Dit paper is vrij technisch, er wordt niet van je verwacht dat je alle details begrijpt, maar het geeft in ieder geval een indruk van de mogelijkheden.
</p>

### Vraag 5

\[2 pt.\]

Implementeer de functie `create_similarity_matrix_jaccard` hieronder. De input is een matrix met een one-hot encoding voor categoriën en de output een similarity matrix gebaseerd op de Jaccard index.

In [15]:
def create_similarity_matrix_jaccard(matrix):
    # TODO
    
    similarity_matrix = pd.DataFrame(data=0, index=matrix.index, columns=matrix.index, dtype=float)
    
    
    for i in range(len(matrix)):
        for j in range(len(matrix)):
            M11 = np.sum((matrix.iloc[i] & matrix.iloc[j]))
            M10 = np.sum((matrix.iloc[i] & (~matrix.iloc[j])))
            M01 = np.sum(((~matrix.iloc[i]) & matrix.iloc[j]))
            
            # Compute Jaccard index and store it in the similarity matrix
            
            similarity_matrix.iloc[i,j] = M11 / (M11 + M10 + M01)
            
    return similarity_matrix
    


test = pd.DataFrame([[0,1,1,0],[1,0,1,0],[0,0,0,0],[1,1,1,1],[0,1,0,1]], index = [18,19,20,21,22])


display(create_similarity_matrix_jaccard(test))

/var/folders/sm/cz_7jnb54j10y39hkp9d7xhm0000gn/T/ipykernel_3083/1700336920.py:15: RuntimeWarning: invalid value encountered in long_scalars
  similarity_matrix.iloc[i,j] = M11 / (M11 + M10 + M01)


,18,19,20,21,22
18,1.000000,0.333333,0.0,0.5,0.333333
19,0.333333,1.000000,0.0,0.5,0.000000
20,0.000000,0.000000,NaN,0.0,0.000000
21,0.500000,0.500000,0.0,1.0,0.500000
22,0.333333,0.000000,0.0,0.5,1.000000


In [16]:
answers.test_5(create_similarity_matrix_jaccard)

Testing create_similarity_matrix_jaccard function: success!


/var/folders/sm/cz_7jnb54j10y39hkp9d7xhm0000gn/T/ipykernel_3083/1700336920.py:15: RuntimeWarning: invalid value encountered in long_scalars
  similarity_matrix.iloc[i,j] = M11 / (M11 + M10 + M01)


Bereken hieronder de similarity matrix voor de `df_utility_genres` utility matrix. 

> Het is erg lastig om `create_similarity_matrix_jaccard` zodanig te optimaliseren dat de onderstaande berekening binnen de perken blijft. Als het uitvoeren van de onderstaande cell erg lang duurt mag je onze oplossing voor de functie gebruiken door de bovenste regel te *uncommenten*.

In [ ]:
## uncomment the line below if your solutions takes too much time to run
# from cf2 import create_similarity_matrix_jaccard

df_similarity_genres = create_similarity_matrix_jaccard(df_utility_genres)
display(df_similarity_genres.head())

Heel veel is er niet veranderd ten opzichte van item-based collaborative filtering: we hebben weer een similarity matrix voor films. Alleen, deze keer is de matrix niet gebaseerd op gebruikersinteractie, maar op film-genres. Zou dit beter werken? Er is maar een manier om erachter te komen.

### Vraag 6

\[1 pt.\]

Gebruik de similarity matrix `df_similarity_genres` met de functie `predict_ratings` voor het voorspellen van filmvoorkeuren en bereken de mean squared error. Sla het resultaat op in de variabele `mse_genres`.

In [ ]:
from cf2 import create_similarity_matrix_jaccard
df_similarity_genres = create_similarity_matrix_jaccard(df_utility_genres)
display(df_similarity_genres.head())

mse_genres = 0

df_predicted_cf_Jac = predict_ratings(df_similarity_genres, df_utility_ratings, actual)
mse_genres = mse(actual, df_predicted_cf_Jac)
# TODO

print(f'mse for content based filtering: {mse_genres:.2f}')

In [ ]:
answers.test_6(mse_genres)

### Vraag 7

\[1 pt.\]

Zonder baseline hadden we ook content based filtering met collaborative filtering kunnen vergelijken. Toch ontbreekt dan informatie die essentieel is voor het trekken van conclusies over de methodes. Welke informatie ontbreekt en welke conclusie kunnen we daardoor niet trekken?

Als we alleen de MSE-waarden hebben voor content-based filtering en item-based collaborative filtering zonder een baseline methode te vergelijken, dan missen we informatie over hoe goed de methodes het doen in vergelijking met een simpele baseline voorspelling, zoals de gemiddelde rating.

Zonder een baseline kunnen we geen onderscheid maken tussen hoeveel beter of slechter een algoritme presteert ten opzichte van een eenvoudige methode, en dus kunnen we geen conclusie trekken over de daadwerkelijke prestaties van de algoritmes.

## Precision-recall curve

Als we naar de mse kijken lijkt het er op dat de content based filtering met film genres veel beter werkt dan collaborative filtering. Maar is dat ook echt zo?

### Vraag 8

\[2 pt.\]

In de vorige opdracht heb je een precision-recall curve gemaakt om wat meer inzicht te krijgen in de prestaties van de algoritmes. Maak hieronder een precision-recall curve waarin je de twee algoritmes die we hierboven hebben geïmplementeerd (item based collaborative filtering en content based filtering op basis van genres) met elkaar vergelijkt. Gebruik weer een vast `treshold_used` en varieer de `treshold_recommended`.

In [ ]:
# TODO
def used(actual, treshold):
    # TODO
    actual = actual[actual['rating']>= treshold]
    return actual.loc[:, actual.columns != 'rating']
    

def unused(actual, treshold):
    # TODO
    actual = actual[actual['rating']< treshold]
    return actual.loc[:, actual.columns != 'rating']
    
def recommended(predictions, treshold):
    # TODOi
    predictions = predictions[predictions['predicted rating'] >= treshold]
    return predictions.loc[:, predictions.columns != 'predicted rating']



def hidden(predictions, treshold):
    # TODO
    predictions = predictions[predictions['predicted rating'] < treshold]
    return predictions.loc[:, predictions.columns != 'predicted rating']

def confusion(recommended, hidden, used, unused):
    TP = len(pd.merge(recommended, used, on=['movieId', 'userId']))
    FP = len(pd.merge(recommended, unused, on=['movieId', 'userId']))
    TN = len(pd.merge(hidden, unused, on=['movieId', 'userId']))
    FN =len(pd.merge(hidden, used, on=['movieId', 'userId']))
    confusion_matrix = pd.DataFrame([[TP, FP], [FN, TN]], columns=['used', 'unused'], index=['recommended', 'hidden'])
    return confusion_matrix

def precision(confusion_matrix):
    TP = confusion_matrix.iloc[0, 0]
    FP = confusion_matrix.iloc[0, 1]
    
    return TP / (TP + FP)
    # TODO
def recall(confusion_matrix):
    TP = confusion_matrix.iloc[0, 0]
    FN = confusion_matrix.iloc[1, 0]
    
    return TP / (TP + FN)
    # TODO

treshold_used = 3.75
treshold_recommended = [2.5,3.0,3.5,4.0,4.5]

used_items = used(actual, treshold_used)
unused_items = unused(actual, treshold_used)


precisionItem = []
recallItem = []

precisionJac = []
recallJac = []

for i in range(len(treshold_recommended)):
    recommendeditem = recommended(df_predicted_cf_item_based, treshold_recommended[i])
    hiddeditem = hidden(df_predicted_cf_item_based, treshold_recommended[i])
    confusionitem = confusion(recommendeditem, hiddeditem, used_items, unused_items)
    precisionItem.append(precision(confusionitem))
    recallItem.append(recall(confusionitem))
    
    recommendedJac = recommended(df_predicted_cf_Jac, treshold_recommended[i])
    hiddedJac = hidden(df_predicted_cf_Jac, treshold_recommended[i])
    confusionJac = confusion(recommendedJac, hiddedJac, used_items, unused_items)
    precisionJac.append(precision(confusionJac))
    recallJac.append(recall(confusionJac))

plt.plot(recallItem, precisionItem)
for r, p, t in zip(recallItem, precisionItem, treshold_recommended):
    plt.text(r, p, t)

plt.plot(recallJac, precisionJac)
for r, p, t in zip(recallItem, precisionItem, treshold_recommended):
    plt.text(r, p, t)


plt.xlim(0.0, 1.0)
plt.ylim(0.0, 1.0)
plt.xlabel('recall')
plt.ylabel('precision')
plt.legend(['itemBased', "Jac"], loc = 'lower left')

Welke aanpak is nou beter? Daar valt wederom geen eenduidig antwoord op te geven. Hetgeen we nu weten is dat _voor deze dataset_ content based een wat hogere precision heeft dan collaborative filtering bij een verglijkbare recall. Maar de verschillen zijn erg klein.

Wat je in zo'n geval zou kunnen doen (*maar wat wij nu niet gaan doen*) is verder onderzoeken wanneer welke aanpak het beste is en wellicht een systeem maken waarin je beide aanpakken op een slimme manier verenigt. Maar er zijn nog zoveel meer bronnen van informatie die we zouden kunnen gebruiken dan alleen genre en ratings. Het zou verstandig zijn om daar eerst eens naar te kijken.